In [1]:
from selenium import webdriver    # 파이썬으로 웹 브라우저 컨트롤
from bs4 import BeautifulSoup as bs  # 분석을 용이하게 정제

In [2]:
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.starbucks.co.kr/store/store_map.do')

In [12]:
# 누를 버튼 좌클릭 > copy카테고리에 copy selector 클릭
area_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'

driver.find_element_by_css_selector(area_btn).click()

In [13]:
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(seoul_btn).click()

In [14]:
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(all_btn).click()

In [16]:
import requests
import pandas as pd

In [17]:
html = driver.page_source # 전체 페이지 소스 읽어오기
soup = bs(html)           # bs로 데이 정제


In [21]:
len(soup.select('ul.quickSearchResultBoxSidoGugun > li'))

569

In [52]:
places = soup.select('ul.quickSearchResultBoxSidoGugun > li')
places[0]
# places[0].select('p.result_details')[0].text

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [92]:

name = places[0]['data-name']   # 지점명
lat = places[0]['data-lat']     # 위도
lng = places[0]['data-long']    # 경도

addr = str(places[0].select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
tel = str(places[0].select('p.result_details')[0]).split('<br/>')[1].split('<')[0] 

In [91]:
places = soup.select('ul.quickSearchResultBoxSidoGugun > li')
data = []
num = 0

for place in places:
    name = place['data-name']   # 지점명
    lat = place['data-lat']     # 위도
    lng = place['data-long']    # 경도
    addr = str(place.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(place.select('p.result_details')[0]).split('<br/>')[1].split('<')[0] 
    num += 1
    data.append([num,name,lat,lng,addr,tel])

In [90]:
df = pd.DataFrame(data, columns = ['번호','지점명','위도','경도','위치','전화번호'])
df

,번호,지점명,위도,경도,위치,전화번호
0,1,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,2,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,3,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,4,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,5,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232
...,...,...,...,...,...,...
564,565,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232
565,566,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
566,567,묵동이마트,37.613433,127.077484,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)",1522-3232
567,568,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232


In [93]:
df.to_excel('Starbucks.xlsx', index=False)

In [95]:
seoul_starbucks = pd.DataFrame(data, columns = ['번호','지점명','위도','경도','위치','전화번호'])

sgg_names = []

for addr in seoul_starbucks['위치']:
    agg = addr.split()[1]
    sgg_names.append(agg)

seoul_starbucks['시군구명'] = sgg_names
seoul_starbucks.head()

,번호,지점명,위도,경도,위치,전화번호,시군구명
0,1,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
1,2,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
2,3,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),1522-3232,강남구
3,4,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,강남구
4,5,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,강남구


In [96]:
starbucks_count = seoul_starbucks.pivot_table(index='시군구명',
                                              values='지점명',
                                             aggfunc='count')

In [97]:
starbucks_count

,지점명
시군구명,
강남구,88
강동구,16
강북구,6
강서구,22
관악구,11
광진구,18
구로구,12
금천구,11
노원구,13


In [124]:
#!pip install folium

import folium

In [125]:
starbucks_map = folium.Map(location=[37.5665, 126.9780], zoom_start=11, tiles='Stamen Terrain')
starbucks_map

In [121]:
# 37.501087	127.043069	

folium.CircleMarker(location=[37.501087, 127.043069]).add_to(starbucks_map)

starbucks_map

In [126]:
for dt in data:
    folium.Marker(location=[dt[2], dt[3]]).add_to(starbucks_map)
starbucks_map